# Fine-tuning XLM-R on CodiEsp-P

In this notebook, following a multi-label sequence classification approach, the XLM-R model is fine-tuned on both the training and development sets of the CodiEsp-P corpus. Additionally, the predictions made by the model on the test set are saved, in order to futher evaluate the clinical coding performance of the model (see `results/CodiEsp-P/Evaluation.ipynb`).

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Auxiliary components
import sys
sys.path.append("..")
from nlp_utils_TF2 import *

# XLM-R tokenizer
from transformers import XLMRobertaTokenizer
import sentencepiece_pb2
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
spt = sentencepiece_pb2.SentencePieceText()

# Hyper-parameters
text_col = "raw_text"
SEQ_LEN = 128
BATCH_SIZE = 16
EPOCHS = 41
LR = 3e-5

random_seed = 0
tf.random.set_seed(random_seed)

## Load text

Firstly, all text files from training and development CodiEsp corpora are loaded in different dataframes.

Also, CIE-Procedimiento codes are loaded.

In [2]:
corpus_path = "../datasets/codiesp_v4/"

### Training corpus

In [3]:
%%time
train_path = corpus_path + "train/text_files/"
train_files = [f for f in os.listdir(train_path) if os.path.isfile(train_path + f)]
train_data = load_text_files(train_files, train_path)
df_text_train = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in train_files], 'raw_text': train_data})

CPU times: user 11.5 ms, sys: 3.62 ms, total: 15.1 ms
Wall time: 14.3 ms


In [4]:
df_text_train.shape

(500, 2)

In [5]:
df_text_train.head()

,doc_id,raw_text
0,S0365-66912006000600011-1,Un varón de 13 años es remitido para valoració...
1,S1139-13752009000200010-2,Paciente de 42 años diagnosticado de pancoliti...
2,S1130-05582017000100037-1,"Varón de 72 años, sin antecedentes médicos de ..."
3,S1139-76322016000300015-1,Lactante de ocho días cuyos padres consultan p...
4,S0211-69952011000100019-1,Mujer de 47 años de edad con antecedentes de g...


In [6]:
df_text_train.raw_text[0]

'Un varón de 13 años es remitido para valoración oftalmológica por mala visión. Fenotípicamente era un niño de talla corta con una estatura de 133 cm, braquimorfia y braquidactilia en las cuatro extremidades.\nEl paciente presentaba un error refractivo corregido de -13,00 -6,50 a 1º en el ojo derecho y de -16,00-6,25 a 179º en el izquierdo. Con dicha corrección alcanzaba una agudeza visual de 0,4 y 0,2 respectivamente. No existía diplopía monocular ni hallazgos en la motilidad ocular extrínseca e intrínseca.\nEl diámetro corneal horizontal era de 12,0 mm en ambos ojos y la paquimetría de 613 y 611 micras respectivamente. La cámara anterior era estrecha, apreciándose iridofacodonesis bilateral. Se evidenció microesferofaquia con desplazamiento anterior de ambos cristalinos dentro de la cámara posterior.\nLa presión intraocular era de 20 mmHg bilateralmente. Gonioscópicamente se apreció un ángulo estrecho simétrico en ambos ojos grado II según Schaffer.\nLa exploración mediante topógrafo

We also load the CIE-Procedimiento codes table:

In [7]:
df_codes_train = pd.read_table(corpus_path + "train/trainP.tsv", sep='\t', header=None)

In [8]:
df_codes_train.columns = ["doc_id", "code"]

In [9]:
df_codes_train.shape

(1550, 2)

In [10]:
df_codes_train.head()

,doc_id,code
0,S0004-06142005000700014-1,bw03zzz
1,S0004-06142005000700014-1,3e02329
2,S0004-06142005000700014-1,bw40zzz
3,S0004-06142005000700014-1,bv44zzz
4,S0004-06142005000700014-1,bn20


In [11]:
len(set(df_codes_train["doc_id"]))

435

### Development corpus

In [12]:
%%time
dev_path = corpus_path + "dev/text_files/"
dev_files = [f for f in os.listdir(dev_path) if os.path.isfile(dev_path + f)]
dev_data = load_text_files(dev_files, dev_path)
df_text_dev = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in dev_files], 'raw_text': dev_data})

CPU times: user 6.61 ms, sys: 72 µs, total: 6.68 ms
Wall time: 5.9 ms


In [13]:
df_text_dev.shape

(250, 2)

In [14]:
df_text_dev.head()

,doc_id,raw_text
0,S1130-63432016000600013-1,"Varón de 75 años, con antecedentes de hiperuri..."
1,S0365-66912003000600010-1,"Paciente de 33 años de edad, gestante de 34 se..."
2,S0211-69952012000200030-1,Mujer de 67 años con múltiples factores de rie...
3,S0365-66912004000900009-1,Paciente de 55 años que acudió a urgencias por...
4,S1139-76322016000300016-2,"Lactante de 1 mes y 29 días, sin antecedentes ..."


In [15]:
df_text_dev.raw_text[0]

'Varón de 75 años, con antecedentes de hiperuricemia en tratamiento con Alopurinol que ingresa para realización de resección transuretral de próstata.\nPostoperatorio inmediato sin incidencias con tratamiento con Pantoprazol, Ciprofloxacino, Paracetamol, Enantyum y Alopurinol. Al cuarto día de postoperatorio presenta mareos, temblor con componente mioclónico en extremidades y tronco e incapacidad para caminar, sin verse alteraciones analíticas. En esta situación se pauta Rivotril y se suspende el tratamiento con Ciprofloxacino, desapareciendo la clínica mioclónica y mejorando el estado del paciente, por lo que se decide el alta hospitalaria.\n\n'

We also load the CIE-Procedimiento codes table:

In [16]:
df_codes_dev = pd.read_table(corpus_path + "dev/devP.tsv", sep='\t', header=None)

In [17]:
df_codes_dev.columns = ["doc_id", "code"]

In [18]:
df_codes_dev.shape

(817, 2)

In [19]:
df_codes_dev.head()

,doc_id,code
0,S0004-06142005000900016-1,bt41zzz
1,S0004-06142005000900016-1,ct13
2,S0004-06142005001000011-1,3e1m39z
3,S0004-06142005001000011-1,0tcb
4,S0004-06142005001000011-1,bt02


In [20]:
len(set(df_codes_dev["doc_id"]))

222

We join the training and development CodiEsp codes dataframes together:

In [21]:
df_codes_train_dev = pd.concat([df_codes_train, df_codes_dev])

In [22]:
df_codes_train_dev.shape

(2367, 2)

In [23]:
df_codes_train_dev.head()

,doc_id,code
0,S0004-06142005000700014-1,bw03zzz
1,S0004-06142005000700014-1,3e02329
2,S0004-06142005000700014-1,bw40zzz
3,S0004-06142005000700014-1,bv44zzz
4,S0004-06142005000700014-1,bn20


## Creating corpora of annotated sentences

Leveraging the information available for the named-entity-recognition and normalization (NER-N) CodiEsp-X task, we create both a training and a development corpus of annotated sentences with CIE-Procedimiento codes.

Firstly, we pre-process the NER-N precedure-codes annotations available for both the training and development corpora.

In [24]:
# Training corpus

In [24]:
%%time

codiesp_x_train = pd.read_table(corpus_path + "train/trainX.tsv", sep='\t', header=None)

CPU times: user 13.8 ms, sys: 0 ns, total: 13.8 ms
Wall time: 12.7 ms


In [25]:
codiesp_x_train.columns = ["doc_id", "type", "code", "word", "location"]

In [26]:
codiesp_x_train.shape

(9181, 5)

In [27]:
codiesp_x_train.head()

,doc_id,type,code,word,location
0,S0004-06142005000700014-1,PROCEDIMIENTO,bw03zzz,Rx tórax,2163 2171
1,S0004-06142005000700014-1,PROCEDIMIENTO,3e02329,Estreptomicina intramuscular,2787 2801;2810 2823
2,S0004-06142005000700014-1,DIAGNOSTICO,n44.8,teste derecho aumentado de tamaño,1343 1376
3,S0004-06142005000700014-1,DIAGNOSTICO,z20.818,exposición a Brucella,594 615
4,S0004-06142005000700014-1,DIAGNOSTICO,r60.9,edemas,1250 1256


In [28]:
codiesp_x_train = codiesp_x_train[codiesp_x_train["type"] == "PROCEDIMIENTO"]

In [29]:
codiesp_x_train.shape

(1972, 5)

In [30]:
df_codes_train_ner = process_ner_labels(codiesp_x_train).sort_values(["doc_id", "start", "end"])

In [31]:
df_codes_train_ner.head()

,doc_id,type,code,word,start,end
0,S0004-06142005000700014-1,PROCEDIMIENTO,bw03zzz,Rx tórax,2163,2171
3,S0004-06142005000700014-1,PROCEDIMIENTO,bw40zzz,Ecografía abdominal,2173,2192
5,S0004-06142005000700014-1,PROCEDIMIENTO,bn20,TAC craneal,2194,2205
4,S0004-06142005000700014-1,PROCEDIMIENTO,bv44zzz,Ecografía testicular,2287,2307
1,S0004-06142005000700014-1,PROCEDIMIENTO,3e02329,Estreptomicina intramuscular,2787,2801


In [32]:
df_codes_train_ner.shape

(2769, 6)

In [34]:
# Development corpus

In [33]:
%%time

codiesp_x_dev = pd.read_table(corpus_path + "dev/devX.tsv", sep='\t', header=None)

CPU times: user 7.51 ms, sys: 0 ns, total: 7.51 ms
Wall time: 6.73 ms


In [34]:
codiesp_x_dev.columns = ["doc_id", "type", "code", "word", "location"]

In [35]:
codiesp_x_dev.shape

(4477, 5)

In [36]:
codiesp_x_dev.head()

,doc_id,type,code,word,location
0,S0004-06142005000900016-1,PROCEDIMIENTO,bt41zzz,ecografía renal derecha,307 316;348 361
1,S0004-06142005000900016-1,PROCEDIMIENTO,ct13,gammagrafía renal,739 756
2,S0004-06142005000900016-1,DIAGNOSTICO,q62.11,estenosis en la unión pieloureteral derecha,540 583
3,S0004-06142005000900016-1,DIAGNOSTICO,n28.89,ectasia pielocalicial,326 347
4,S0004-06142005000900016-1,DIAGNOSTICO,n39.0,infecciones del tracto urinario,198 229


In [37]:
codiesp_x_dev = codiesp_x_dev[codiesp_x_dev["type"] == "PROCEDIMIENTO"]

In [38]:
codiesp_x_dev.shape

(1046, 5)

In [39]:
df_codes_dev_ner = process_ner_labels(codiesp_x_dev).sort_values(["doc_id", "start", "end"])

In [40]:
df_codes_dev_ner.head()

,doc_id,type,code,word,start,end
0,S0004-06142005000900016-1,PROCEDIMIENTO,bt41zzz,ecografía renal derecha,307,316
1,S0004-06142005000900016-1,PROCEDIMIENTO,bt41zzz,ecografía renal derecha,348,361
2,S0004-06142005000900016-1,PROCEDIMIENTO,ct13,gammagrafía renal,739,756
3,S0004-06142005001000011-1,PROCEDIMIENTO,3e1m39z,diálisis peritoneal,95,114
7,S0004-06142005001000011-1,PROCEDIMIENTO,0270,angioplastia transluminal de la coronaria derecha,424,473


In [41]:
df_codes_dev_ner.shape

(1540, 6)

Now, using the character start-end positions of each sentence from the CodiEsp corpus (see `datasets/CodiEsp-Sentence-Split.ipynb`), we annotate the sentences with CIE-Procedimiento codes. Also, using XLM-R tokenizer, each sentence is converted into a sequence of subwords, which are further converted into vocabulary indices (input IDs) and attention mask arrays (XLM-R input tensors).

In [42]:
# Sentence-Split information
ss_corpus_path = "../datasets/CodiEsp-SSplit-text_SE_All_Pos/"

### Training corpus

In [43]:
label_list = list(df_codes_train_dev["code"])

In [44]:
len(label_list)

2367

In [45]:
len(set(label_list))

727

In [46]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb_encoder = MultiLabelBinarizer()
mlb_encoder.fit([label_list])

MultiLabelBinarizer()

In [47]:
# Number of distinct codes
num_labels = len(mlb_encoder.classes_)

In [48]:
num_labels

727

Only training texts that are annotated with CIE-Procedimiento codes are considered:

In [49]:
# Some train documents (texts) are not annotated 
len(set(df_text_train["doc_id"]) - set(df_codes_train_ner["doc_id"]))

65

In [50]:
train_doc_list = sorted(set(df_codes_train_ner["doc_id"]))

In [51]:
len(train_doc_list)

435

In [54]:
# Sentence-Split data

In [52]:
%%time
ss_sub_corpus_path = ss_corpus_path + "train/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_train = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 18.3 ms, sys: 0 ns, total: 18.3 ms
Wall time: 17.5 ms


In [53]:
%%time
train_ind, train_att, train_y, train_frag, train_start_end_frag = ss_brute_force_create_frag_input_data(df_text=df_text_train, 
                                                  text_col=text_col, 
                                                  df_ann=df_codes_train_ner, doc_list=train_doc_list, ss_dict=ss_dict_train,
                                                  tokenizer=tokenizer, sp_pb2=spt, lab_encoder=mlb_encoder, seq_len=SEQ_LEN)

100%|██████████| 435/435 [00:04<00:00, 95.27it/s] 


CPU times: user 4.68 s, sys: 19.3 ms, total: 4.7 s
Wall time: 4.67 s


In [57]:
# Sanity check

In [54]:
train_ind.shape

(7013, 128)

In [55]:
train_att.shape

(7013, 128)

In [56]:
train_y.shape

(7013, 727)

In [57]:
len(train_frag)

435

In [58]:
len(train_start_end_frag)

7013

In [59]:
# Check n_frag distribution across texts
pd.Series(train_frag).describe()

count    435.000000
mean      16.121839
std        7.762687
min        4.000000
25%       10.500000
50%       15.000000
75%       20.000000
max       54.000000
dtype: float64

In [60]:
# Inspect a randomly selected text and its encoded version
check_id = np.random.randint(low=0, high=len(train_doc_list), size=1)[0]

In [61]:
check_id

177

In [62]:
train_doc_list[check_id]

'S0365-66912007000200010-1'

In [63]:
df_text_train[df_text_train["doc_id"] == train_doc_list[check_id]][text_col].values[0]

'Mujer de 45 años que es remitida desde el servicio de neurocirugía por pérdida de visión súbita bilateral tras cirugía de derivación ventrículoperitoneal de un higroma subdural temporal y absceso occipital craneal. Todo ello resultado de una complicación postquirúrgica subaguda. Quince días antes había sido intervenida de descompresión preventiva medular por Malformación de Arnold-Chiari I.\nLa paciente contaba con una clínica de 16 años de evolución, con cefaleas de intensidad leve, inestabilidad, mareos rotatorios, vómitos autolimitados de duración variable y nistagmus vertical leve desde hacía un año. Éste último ayudó a la sospecha y diagnóstico de la malformación congénita, ya que el resto de su sintomatología la relacionaba con la colitis ulcerosa que sufría hacía años.\nLa agudeza visual (AV) en la primera exploración fue de movimiento de manos en ambos ojos. Las reacciones pupilares fueron de 2+ sin defecto pupilar aferente relativo. Presentaba un nistagmus en sacudida hacia a

In [64]:
check_id_frag = sum(train_frag[:check_id])
for i in range(check_id_frag, check_id_frag + train_frag[check_id]):
    print(mlb_encoder.inverse_transform(np.array([train_y[i]])), "\n")

[('0016',)] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[('08j0xzz', '08j1xzz', 'b847zzz')] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 



In [65]:
for i in range(check_id_frag, check_id_frag + train_frag[check_id]):
    print(list(zip([tokenizer._convert_id_to_token(int(ind)) for ind in train_ind[i]][1:len(train_start_end_frag[i])+1], 
               train_start_end_frag[i])))
    print("\n")

[('▁Mu', (0, 2)), ('jer', (2, 5)), ('▁de', (5, 8)), ('▁45', (8, 11)), ('▁años', (11, 17)), ('▁que', (17, 21)), ('▁es', (21, 24)), ('▁remit', (24, 30)), ('ida', (30, 33)), ('▁desde', (33, 39)), ('▁el', (39, 42)), ('▁servicio', (42, 51)), ('▁de', (51, 54)), ('▁neuro', (54, 60)), ('ci', (60, 62)), ('rug', (62, 65)), ('ía', (65, 68)), ('▁por', (68, 72)), ('▁pérdida', (72, 81)), ('▁de', (81, 84)), ('▁visión', (84, 92)), ('▁sú', (92, 96)), ('bita', (96, 100)), ('▁bilateral', (100, 110)), ('▁tras', (110, 115)), ('▁cirugía', (115, 124)), ('▁de', (124, 127)), ('▁deriva', (127, 134)), ('ción', (134, 139)), ('▁vent', (139, 144)), ('rí', (144, 147)), ('culo', (147, 151)), ('peri', (151, 155)), ('tone', (155, 159)), ('al', (159, 161)), ('▁de', (161, 164)), ('▁un', (164, 167)), ('▁hi', (167, 170)), ('gro', (170, 173)), ('ma', (173, 175)), ('▁sub', (175, 179)), ('dur', (179, 182)), ('al', (182, 184)), ('▁temporal', (184, 193)), ('▁y', (193, 195)), ('▁abs', (195, 199)), ('ces', (199, 202)), ('o', (202

In [66]:
check_id_frag = sum(train_frag[:check_id])
for frag in train_ind[check_id_frag:check_id_frag + train_frag[check_id]]:
    print(' '.join([tokenizer._convert_id_to_token(int(ind)) for ind in frag]), "\n")

<s> ▁Mu jer ▁de ▁45 ▁años ▁que ▁es ▁remit ida ▁desde ▁el ▁servicio ▁de ▁neuro ci rug ía ▁por ▁pérdida ▁de ▁visión ▁sú bita ▁bilateral ▁tras ▁cirugía ▁de ▁deriva ción ▁vent rí culo peri tone al ▁de ▁un ▁hi gro ma ▁sub dur al ▁temporal ▁y ▁abs ces o ▁oc cipit al ▁c rane al . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

<s> ▁Todo ▁ello ▁resultado ▁de ▁una ▁complica ción ▁post qui rú r gica ▁sub a guda . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

In [67]:
# Fragment labels distribution
pd.Series(np.sum(train_y, axis=1)).describe()

count    7013.000000
mean        0.305290
std         0.652715
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         6.000000
dtype: float64

### Development corpus

Only development texts that are annotated with CIE-Procedimiento codes are considered:

In [68]:
# Some dev documents (texts) are not annotated 
len(set(df_text_dev["doc_id"]) - set(df_codes_dev_ner["doc_id"]))

28

In [69]:
dev_doc_list = sorted(set(df_codes_dev_ner["doc_id"]))

In [70]:
len(dev_doc_list)

222

In [71]:
%%time
ss_sub_corpus_path = ss_corpus_path + "dev/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_dev = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 10.9 ms, sys: 0 ns, total: 10.9 ms
Wall time: 10.1 ms


In [72]:
%%time
dev_ind, dev_att, dev_y, dev_frag, dev_start_end_frag = ss_brute_force_create_frag_input_data(df_text=df_text_dev, 
                                                  text_col=text_col, 
                                                  df_ann=df_codes_dev_ner, doc_list=dev_doc_list, ss_dict=ss_dict_dev,
                                                  tokenizer=tokenizer, sp_pb2=spt, lab_encoder=mlb_encoder, seq_len=SEQ_LEN)

100%|██████████| 222/222 [00:02<00:00, 86.88it/s] 


CPU times: user 2.63 s, sys: 3.31 ms, total: 2.63 s
Wall time: 2.61 s


In [77]:
# Sanity check

In [73]:
dev_ind.shape

(3799, 128)

In [74]:
dev_att.shape

(3799, 128)

In [75]:
dev_y.shape

(3799, 727)

In [76]:
len(dev_frag)

222

In [77]:
len(dev_start_end_frag)

3799

In [78]:
# Check n_frag distribution across texts
pd.Series(dev_frag).describe()

count    222.000000
mean      17.112613
std        8.320553
min        4.000000
25%       11.000000
50%       15.000000
75%       21.000000
max       65.000000
dtype: float64

In [79]:
# Inspect a randomly selected text and its encoded version
check_id = np.random.randint(low=0, high=len(dev_doc_list), size=1)[0]

In [80]:
check_id

27

In [81]:
dev_doc_list[check_id]

'S0210-48062003001000009-1'

In [82]:
df_text_dev[df_text_dev["doc_id"] == dev_doc_list[check_id]][text_col].values[0]

'Una mujer de 35 años consultó por goteo terminal o "micción babeante", dispareunia e infecciones urinarias de repetición. En la urografía intravenosa y en la cistografía miccional se encontraron hallazgos compatibles con divertículo uretral. La paciente aquejaba frecuencia urinaria y nicturia de 3 a 4 veces. Ocasionalmente había tenido episodios aislados de incontinencia urinaria de estrés por lo que usaba una compresa diaria de protección. Presentaba urgencia urinaria e incontinencia de urgencia ocasional. No presentaba micción entrecortada, ni sensación de presión vesical o de vaciamiento incompleto. No presentaba hematuria ni síntomas ni signos de cáncer genitourinario. Ritmo intestinal normal. No disfunción sexual femenina previa hasta la presente dispareunia que afecta a su vida sexual. No antecedentes patológicos neurológicos.\nAntecedentes médicos y quirúrgicos: Hipertensión arterial. Asma en la infancia. Dolor lumbosacro ocasional. Colecistectomía laparoscópica hace 2 años. Do

In [83]:
check_id_frag = sum(dev_frag[:check_id])
for i in range(check_id_frag, check_id_frag + dev_frag[check_id]):
    print(mlb_encoder.inverse_transform(np.array([dev_y[i]])), "\n")

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[('10d0',)] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[('br3c',)] 

[()] 

[()] 

[()] 

[()] 

[()] 

[('0tjb8zz',)] 

[()] 

[('0tjb8zz',)] 

[()] 

[()] 

[()] 

[('0tjb8zz',)] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[('0t1b',)] 

[()] 

[()] 

[('0tjb8zz',)] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[('0t1b',)] 

[()] 

[()] 



In [84]:
for i in range(check_id_frag, check_id_frag + dev_frag[check_id]):
    print(list(zip([tokenizer._convert_id_to_token(int(ind)) for ind in dev_ind[i]][1:len(dev_start_end_frag[i])+1], 
               dev_start_end_frag[i])))
    print("\n")

[('▁Una', (0, 3)), ('▁mujer', (3, 9)), ('▁de', (9, 12)), ('▁35', (12, 15)), ('▁años', (15, 21)), ('▁consult', (21, 29)), ('ó', (29, 31)), ('▁por', (31, 35)), ('▁got', (35, 39)), ('eo', (39, 41)), ('▁terminal', (41, 50)), ('▁o', (50, 52)), ('▁"', (52, 54)), ('mi', (54, 56)), ('cción', (56, 62)), ('▁babe', (62, 67)), ('ante', (67, 71)), ('",', (71, 73)), ('▁dispar', (73, 80)), ('eu', (80, 82)), ('nia', (82, 85)), ('▁e', (85, 87)), ('▁in', (87, 90)), ('fec', (90, 93)), ('ciones', (93, 99)), ('▁urin', (99, 104)), ('arias', (104, 109)), ('▁de', (109, 112)), ('▁repeti', (112, 119)), ('ción', (119, 124)), ('.', (124, 125))]


[('▁En', (126, 128)), ('▁la', (128, 131)), ('▁ur', (131, 134)), ('ografía', (134, 142)), ('▁intra', (142, 148)), ('ven', (148, 151)), ('osa', (151, 154)), ('▁y', (154, 156)), ('▁en', (156, 159)), ('▁la', (159, 162)), ('▁ci', (162, 165)), ('st', (165, 167)), ('ografía', (167, 175)), ('▁mic', (175, 179)), ('cional', (179, 185)), ('▁se', (185, 188)), ('▁encontrar', (188, 19

In [85]:
check_id_frag = sum(dev_frag[:check_id])
for frag in dev_ind[check_id_frag:check_id_frag + dev_frag[check_id]]:
    print(' '.join([tokenizer._convert_id_to_token(int(ind)) for ind in frag]), "\n")

<s> ▁Una ▁mujer ▁de ▁35 ▁años ▁consult ó ▁por ▁got eo ▁terminal ▁o ▁" mi cción ▁babe ante ", ▁dispar eu nia ▁e ▁in fec ciones ▁urin arias ▁de ▁repeti ción . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

<s> ▁En ▁la ▁ur ografía ▁intra ven osa ▁y ▁en ▁la ▁ci st ografía ▁mic cional ▁se ▁encontrar on ▁halla z gos ▁compatible s ▁con ▁di ver tí culo ▁ure tral . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad

In [86]:
# Fragment labels distribution
pd.Series(np.sum(dev_y, axis=1)).describe()

count    3799.000000
mean        0.309818
std         0.638081
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         5.000000
dtype: float64

### Training & Development corpus

We merge the previously generated datasets:

In [87]:
# Indices
train_dev_ind = np.concatenate((train_ind, dev_ind))

In [88]:
train_dev_ind.shape

(10812, 128)

In [89]:
# Attention masks
train_dev_att = np.concatenate((train_att, dev_att))

In [90]:
train_dev_att.shape

(10812, 128)

In [91]:
# y
train_dev_y = np.concatenate((train_y, dev_y))

In [92]:
train_dev_y.shape

(10812, 727)

## Fine-tuning

Using the corpus of labeled sentences, we fine-tune the model on a multi-label sentence classification task.

In [98]:
from transformers import TFXLMRobertaForSequenceClassification

model = TFXLMRobertaForSequenceClassification.from_pretrained(model_name, from_pt=True)

All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [99]:
model.summary()

Model: "tfxlm_roberta_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  277453056 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  592130    
Total params: 278,045,186
Trainable params: 278,045,186
Non-trainable params: 0
_________________________________________________________________


In [100]:
model.layers

In [102]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.initializers import GlorotUniform

input_ids = Input(shape=(SEQ_LEN,), name='input_ids', dtype='int64')
attention_mask = Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int64')
inputs = [input_ids, attention_mask]

cls_token = model.layers[0](input_ids=inputs[0], attention_mask=inputs[1])[0][:, 0, :] # take <s> token output representation (equiv. to [CLS]) 
out_logits = Dense(units=num_labels, kernel_initializer=GlorotUniform(seed=random_seed))(cls_token) # Multi-label classification
out_act = Activation('sigmoid')(out_logits)

model = Model(inputs=[input_ids, attention_mask], outputs=out_act)

In [103]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
roberta (TFRobertaMainLayer)    TFBaseModelOutputWit 277453056   input_ids[0][0]                  
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 768)]        0           roberta[0][0]                    
______________________________________________________________________________________________

In [104]:
model.input

[<tf.Tensor 'input_ids:0' shape=(None, 128) dtype=int64>,
 <tf.Tensor 'attention_mask:0' shape=(None, 128) dtype=int64>]

In [105]:
model.output

<tf.Tensor 'activation_4/Identity:0' shape=(None, 727) dtype=float32>

In [ ]:
%%time
from tensorflow.keras import optimizers, losses
import tensorflow_addons as tfa

optimizer = tfa.optimizers.RectifiedAdam(learning_rate=LR)
loss = losses.BinaryCrossentropy(from_logits=False)
model.compile(optimizer=optimizer, loss=loss)

history = model.fit(x={'input_ids': train_dev_ind, 'attention_mask': train_dev_att}, y=train_dev_y,
          batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=True)

Epoch 1/41


## Test set predictions

Finally, the predictions made by the model on the test set are saved. For this purpose, firstly, each sentence from the test corpus must be converted into a sequence of subwords (input IDs and attention mask arrays). Then, the predictions made by the model at the sentence-level are saved, to be further evaluated at document-level (see `results/CodiEsp-P/Evaluation.ipynb`).

In [101]:
%%time
test_path = corpus_path + "test/text_files/"
test_files = [f for f in os.listdir(test_path) if os.path.isfile(test_path + f)]
test_data = load_text_files(test_files, test_path)
df_text_test = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in test_files], 'raw_text': test_data})

CPU times: user 8.68 ms, sys: 131 µs, total: 8.81 ms
Wall time: 7.84 ms


In [102]:
df_text_test.shape

(250, 2)

In [103]:
df_text_test.head()

,doc_id,raw_text
0,S0365-66912007000900014-1,Paciente varón de 34 años de edad diagnosticad...
1,S0211-69952014000200012-1,"Un varón de 48 años, de raza caucásica, con IR..."
2,S1139-76322017000200009-1,Presentamos el caso clínico de un niño de cinc...
3,S0210-48062010000100019-1,"Paciente varón de 53 años, diagnosticado de es..."
4,S1130-14732005000500006-1,Se trata de un varón de 20 años diagnosticado ...


In [104]:
df_text_test.raw_text[0]

'Paciente varón de 34 años de edad diagnosticado de varicela tres semanas antes ya resuelta sin complicaciones. Acude a urgencias por presentar disminución de agudeza visual en su ojo izquierdo.\nEn la exploración oftalmológica presenta una agudeza visual corregida de 1 en el ojo derecho (OD) y de 0,6 en el ojo izquierdo (OI). El estudio con lámpara de hendidura demuestra en el OI un tyndall celular de 4+, precipitados queráticos inferiores (3+) y sin presentar la cornea tinción con fluoresceína, siendo normal el OD. La presión intraocular fue de 16mmHg en ambos ojos.\nEn la exploración fundoscópica inicial del OI se aprecia leve vitritis (1+) sin focos de retinitis.\nSe instaura tratamiento tópico con corticoides y midriáticos. A los 2 días se observa leve disminución del tyndall celular (3+) en cámara anterior pero en fondo de ojo aparece un foco periférico de retinitis necrotizante en el área temporal asociado a vasculitis retiniana.\nSe ingresa al paciente y se instaura tratamiento

In [105]:
test_doc_list = sorted(set(df_text_test["doc_id"]))

In [106]:
len(test_doc_list)

250

In [107]:
%%time
ss_sub_corpus_path = ss_corpus_path + "test/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_test = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 6.22 ms, sys: 3.85 ms, total: 10.1 ms
Wall time: 9.31 ms


In [108]:
%%time
test_ind, test_att, _, test_frag, _ = ss_brute_force_create_frag_input_data(df_text=df_text_test, 
                                                  text_col=text_col, 
                                                  df_ann=df_codes_train_ner, doc_list=test_doc_list, ss_dict=ss_dict_test,
                                                  tokenizer=tokenizer, sp_pb2=spt, lab_encoder=mlb_encoder, seq_len=SEQ_LEN)

100%|██████████| 250/250 [00:01<00:00, 248.26it/s]

CPU times: user 1.08 s, sys: 8.43 ms, total: 1.09 s
Wall time: 1.07 s


In [173]:
%%time
test_preds = model.predict({'input_ids': test_ind, 'attention_mask': test_att})

CPU times: user 14.3 s, sys: 2.04 s, total: 16.3 s
Wall time: 19.3 s


In [174]:
test_preds.shape

(3950, 727)

In [109]:
results_dir_path = "../results/CodiEsp-P/"

In [178]:
%%time
np.save(file=results_dir_path + "predictions/xlm_r_seed_" + str(random_seed) + "_test_preds.npy", arr=test_preds)

CPU times: user 2.21 ms, sys: 4.65 ms, total: 6.87 ms
Wall time: 6.02 ms


In [111]:
# To be further used when evaluating model performance at document level
np.save(file=results_dir_path + "xlm_r_test_frags.npy", arr=test_frag)
np.save(file=results_dir_path + "classes.npy", arr=mlb_encoder.classes_)